In [ ]:
#!python3 -m pip install --upgrade langchain deeplake openai tiktoken

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()
# Please manually enter OpenAI Key

In [ ]:
os.environ["ACTIVELOOP_TOKEN"] = getpass()

In [ ]:
DEEPLAKE_ACCOUNT_NAME = "ngmisl"

In [ ]:
from langchain.document_loaders import WebBaseLoader


In [ ]:
loader = WebBaseLoader("https://docs.modular.com/mojo/programming-manual.html")
docs = loader.load()


In [ ]:
from langchain.text_splitter import CharacterTextSplitter


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
print(f"{len(texts)}")

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings


In [ ]:
from langchain.vectorstores import DeepLake

db = DeepLake.from_documents(
    texts, embeddings, dataset_path=f"hub://{DEEPLAKE_ACCOUNT_NAME}/mojo-docs"
)
db

In [ ]:
db = DeepLake(
    dataset_path=f"hub://{DEEPLAKE_ACCOUNT_NAME}/mojo-docs",
    read_only=True,
    embedding_function=embeddings,
)

In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model="gpt-3.5-turbo")  # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [37]:
questions = [
    "What is the difference between def and fn?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is the difference between def and fn? 

**Answer**: From an interface level, def and fn are always interchangeable, meaning that there is nothing a ‘def’ can provide that a fn cannot (or vice versa). However, the difference between the two is that a fn is more limited and controlled on the inside of its body. Fns have a number of limitations compared to defs, such as more strict scoping, explicit type annotations, and immutability by default for argument passing. Fns are designed to provide more control and predictability for systems programming, while defs are more flexible and dynamic for high-level programming and scripting. It is up to the programmer to decide which to use based on their use-case. 



**Answer**: From an interface level, def and fn are always interchangeable, meaning that there is nothing a ‘def’ can provide that a fn cannot (or vice versa). However, the difference between the two is that a fn is more limited and controlled on the inside of its body. Fns have a number of limitations compared to defs, such as more strict scoping, explicit type annotations, and immutability by default for argument passing. Fns are designed to provide more control and predictability for systems programming, while defs are more flexible and dynamic for high-level programming and scripting. It is up to the programmer to decide which to use based on their use-case.